In [1]:
#Potrebne biblioteke
import numpy as np
import pandas as pd 
import os
import shutil

### 1. Generiranje dokumenta za označavanje aktivnosti

Ovaj dio skripte je potrebno pokrenuti kada se kreće sa postupkom označavanja.\
U slučaju da već postoji dokument sa oznakama ovo nije potrebno pokrenuti !!

In [2]:
#Putanje do uzoraka u različitim kadrovima
path_HE = "C:/Users/Public/Uzorak_slika/HE"
path_Fine = "C:/Users/Public/Uzorak_slika/Fine"
path_Fokus = "C:/Users/Public/Uzorak_slika/Fokus"

In [3]:
#Lista uzoraka
sample_path = sorted(os.listdir(path_HE), key=lambda x: int(x.split("_")[0]))#trik da uzorci budu poredani prema prvom elementu oznake

In [4]:
#Generiranje .txt datoteke sa svim podatcima za označavanje - PRAZAN
with open("C:/Users/Public/Labels/labels_empty.txt", "w") as labels:
    labels.write("Video_ID,Activity_ID,Activity_name,Start_time,End_time,Start_frame,End_frame\n")
    for label_id, path in enumerate(sample_path): 
        #Ovo je moj hack, jer znam da aktivnosti idu ovim ciklusom, te će mi ovo ubrzati označavanje
        labels.writelines([f"{path},1,Formiranje kuta,'','','',''\n",
                           f"{path},2,Umetanje i ucvrscivanje kopce,'','','',''\n",
                           f"{path},3,Umetanje lamela,'','','',''\n",
                           f"{path},4,Postavljanje poprecne stranice,'','','',''\n",
                           f"{path},2,Umetanje i ucvrscivanje kopce,'','','',''\n",
                           f"{path},5,Postavljanje uzduzne stranice,'','','',''\n",
                           f"{path},2,Umetanje i ucvrscivanje kopce,'','','',''\n",
                           f"{path},2,Umetanje i ucvrscivanje kopce,'','','',''\n",
                           f"{path},6,Odlaganje gotovog proizvoda,'','','',''\n"])

In [10]:
#Provjera 1
with open("C:/Users/Public/Labels/labels_empty.txt", "r") as labels:
    df = labels.readlines()

In [5]:
#Provjera 2
df_2 = pd.read_csv("C:/Users/Public/Labels/labels_empty.txt")

In [6]:
#Prebacivanje iz .txt u .xlsx, jer je u njemu lakše i brže označavati
df_2.to_excel("C:/Users/Public/Labels/labels_empty.xlsx", index=False) #index=False izbacuje prvu kolonu koja daje numeraciju 

### 2. Dodavanje vremena početka i završetka aktivnosti u tablicu oznaka

Nakon što je napravljeno označavanje start i end sličica, potrebno je pokrenuti ovaj dio skripte.\
U slučaju da su u oznake već dodana vremena nije potrebno pokretati ovaj dio skripte!!

In [7]:
#Učitavanje podataka kod koji su označene samo sličice, bez oznaka vremena, samo 6 aktivnosti, 620 uzoraka
path_frame_labs = "C:/Users/Public/Labels/labels_no_time.xlsx"
frame_labs = pd.read_excel(path_frame_labs, header=0)

In [8]:
frame_labs

,Video_ID,Activity_ID,Activity_name,Start_time,End_time,Start_frame,End_frame
0,1_D1_V1_O1_T1,1,Formiranje kuta,'','',6,26
1,1_D1_V1_O1_T1,2,Umetanje i ucvrscivanje kopce,'','',27,59
2,1_D1_V1_O1_T1,3,Umetanje lamela,'','',65,112
3,1_D1_V1_O1_T1,4,Postavljanje poprecne stranice,'','',119,145
4,1_D1_V1_O1_T1,2,Umetanje i ucvrscivanje kopce,'','',146,176
...,...,...,...,...,...,...,...
5575,628_D7_V22_O3_T1,2,Umetanje i ucvrscivanje kopce,'','',147,182
5576,628_D7_V22_O3_T1,5,Postavljanje uzduzne stranice,'','',184,197
5577,628_D7_V22_O3_T1,2,Umetanje i ucvrscivanje kopce,'','',199,236
5578,628_D7_V22_O3_T1,2,Umetanje i ucvrscivanje kopce,'','',243,271


In [9]:
#Provjera tipa podataka u kolonama
frame_labs.dtypes 

Video_ID         object
Activity_ID       int64
Activity_name    object
Start_time       object
End_time         object
Start_frame       int64
End_frame         int64
dtype: object

In [10]:
#Izračun početka i kraja aktivnosti na temelju oznake sličica
labs_all = frame_labs.assign(Start_time=lambda x: x['Start_frame'] * 0.2 - 0.2, #* 0.2 jer je napravljeno poduzorkovanja sa fps=5
                             End_time=lambda x: x['End_frame'] * 0.2 - 0.2) # -0.2, jer želimo da je početna sličica t=0

labs_all

,Video_ID,Activity_ID,Activity_name,Start_time,End_time,Start_frame,End_frame
0,1_D1_V1_O1_T1,1,Formiranje kuta,1.0,5.0,6,26
1,1_D1_V1_O1_T1,2,Umetanje i ucvrscivanje kopce,5.2,11.6,27,59
2,1_D1_V1_O1_T1,3,Umetanje lamela,12.8,22.2,65,112
3,1_D1_V1_O1_T1,4,Postavljanje poprecne stranice,23.6,28.8,119,145
4,1_D1_V1_O1_T1,2,Umetanje i ucvrscivanje kopce,29.0,35.0,146,176
...,...,...,...,...,...,...,...
5575,628_D7_V22_O3_T1,2,Umetanje i ucvrscivanje kopce,29.2,36.2,147,182
5576,628_D7_V22_O3_T1,5,Postavljanje uzduzne stranice,36.6,39.2,184,197
5577,628_D7_V22_O3_T1,2,Umetanje i ucvrscivanje kopce,39.6,47.0,199,236
5578,628_D7_V22_O3_T1,2,Umetanje i ucvrscivanje kopce,48.4,54.0,243,271


In [11]:
#Provjera tipa podataka u kolonama "labs_all"
labs_all.dtypes

Video_ID          object
Activity_ID        int64
Activity_name     object
Start_time       float64
End_time         float64
Start_frame        int64
End_frame          int64
dtype: object

#### Prelazak iz 6 aktivnosti u 9 aktivnosti

Ova tablica se razlikuje sa aspekta označavanja aktivnosti. Konkretno aktivnost broj 2 dijelimo na četiri različite aktivnosti, tj. svaki kut ima svoju oznaku:
* 2 => Umetanje i ucvrscivanje kopce_LG
* 7 => Umetanje i ucvrscivanje kopce_DG
* 8 => Umetanje i ucvrscivanje kopce_DD
* 9 => Umetanje i ucvrscivanje kopce_LD

In [12]:
#Kreiranje alternativnih oznaka
path_data = "C:/Users/Public/Uzorak_slika/HE"
sample_path = os.listdir(path_data)

with open("C:/Users/Public/Labels/alternate_activity_labels.txt", "w") as activities:
    activities.write("Video_ID,Activity_ID,Activity_name\n")
    for label_id, path in enumerate(sorted(sample_path, key=lambda x: int(x.split("_")[0]))): #trik da uzorci budu poredani prema prvom elementu oznake
        activities.writelines([f"{path},1,Formiranje kuta\n",
                               f"{path},2,Umetanje i ucvrscivanje kopce_LG\n",
                               f"{path},3,Umetanje lamela\n",
                               f"{path},4,Postavljanje poprecne stranice\n",
                               f"{path},7,Umetanje i ucvrscivanje kopce_DG\n",
                               f"{path},5,Postavljanje uzduzne stranice\n",
                               f"{path},8,Umetanje i ucvrscivanje kopce_DD\n",
                               f"{path},9,Umetanje i ucvrscivanje kopce_LD\n",
                               f"{path},6,Odlaganje gotovog proizvoda\n"])

In [13]:
#Dodavanje alternativnih oznaka
alt_activities = pd.read_csv("C:/Users/Public/Labels/alternate_activity_labels.txt", header=0)
alt_activities

,Video_ID,Activity_ID,Activity_name
0,1_D1_V1_O1_T1,1,Formiranje kuta
1,1_D1_V1_O1_T1,2,Umetanje i ucvrscivanje kopce_LG
2,1_D1_V1_O1_T1,3,Umetanje lamela
3,1_D1_V1_O1_T1,4,Postavljanje poprecne stranice
4,1_D1_V1_O1_T1,7,Umetanje i ucvrscivanje kopce_DG
...,...,...,...
5575,628_D7_V22_O3_T1,7,Umetanje i ucvrscivanje kopce_DG
5576,628_D7_V22_O3_T1,5,Postavljanje uzduzne stranice
5577,628_D7_V22_O3_T1,8,Umetanje i ucvrscivanje kopce_DD
5578,628_D7_V22_O3_T1,9,Umetanje i ucvrscivanje kopce_LD


In [14]:
#Dodane alternativne oznake
labs_all['Activity_ID'] = alt_activities['Activity_ID']
labs_all['Activity_name'] = alt_activities['Activity_name']
labs_all

,Video_ID,Activity_ID,Activity_name,Start_time,End_time,Start_frame,End_frame
0,1_D1_V1_O1_T1,1,Formiranje kuta,1.0,5.0,6,26
1,1_D1_V1_O1_T1,2,Umetanje i ucvrscivanje kopce_LG,5.2,11.6,27,59
2,1_D1_V1_O1_T1,3,Umetanje lamela,12.8,22.2,65,112
3,1_D1_V1_O1_T1,4,Postavljanje poprecne stranice,23.6,28.8,119,145
4,1_D1_V1_O1_T1,7,Umetanje i ucvrscivanje kopce_DG,29.0,35.0,146,176
...,...,...,...,...,...,...,...
5575,628_D7_V22_O3_T1,7,Umetanje i ucvrscivanje kopce_DG,29.2,36.2,147,182
5576,628_D7_V22_O3_T1,5,Postavljanje uzduzne stranice,36.6,39.2,184,197
5577,628_D7_V22_O3_T1,8,Umetanje i ucvrscivanje kopce_DD,39.6,47.0,199,236
5578,628_D7_V22_O3_T1,9,Umetanje i ucvrscivanje kopce_LD,48.4,54.0,243,271


In [15]:
#Spremanje oznaka u .xlsx formatu i .txt formatu
labs_all.to_excel("C:/Users/Public/Labels/labels_all.xlsx", index=False) #index=False izbacuje prvu kolonu koja daje numeraciju 
labs_all.to_csv("C:/Users/Public/Labels/labels_all.txt",float_format='%.1f', index=False) #definirano je jedno decimalno mjesto 

### 3. Kreiranje dodatne tablice za statističku obradu

#### Tablica za statističku obradu

In [16]:
labs_stat = labs_all.copy()
#Uklanjanje kolona sa oznakama sličica
labs_stat = labs_all.drop(['Start_frame', 'End_frame'], axis=1)
#Dodavanje kolone "Duration" koja je razlika između "End_time" i "Start_time"
labs_stat['Duration'] = labs_stat['End_time'] - labs_stat['Start_time']
#Dodavanje oznaka operatera ("Subject") i tipa proizvoda("Product_type") i oznake uzorka ("Sample_ID") izvedenih iz kolone "Video_ID"
labs_stat[['Sample_num', 'Day_ID', 'Video_num', 'Subject', 'Product_type']] = labs_stat['Video_ID'].str.split('_', expand=True) #expand tako da vrati DataFrame inače vraća series
#Odbaci kolone Day_ID, Video_num
labs_stat = labs_stat.drop(['Day_ID', 'Video_num'], axis=1)
#Spremanje
labs_stat.to_excel("C:/Users/Public/Labels/labels_stat.xlsx", index=False)
labs_stat

,Video_ID,Activity_ID,Activity_name,Start_time,End_time,Duration,Sample_num,Subject,Product_type
0,1_D1_V1_O1_T1,1,Formiranje kuta,1.0,5.0,4.0,1,O1,T1
1,1_D1_V1_O1_T1,2,Umetanje i ucvrscivanje kopce_LG,5.2,11.6,6.4,1,O1,T1
2,1_D1_V1_O1_T1,3,Umetanje lamela,12.8,22.2,9.4,1,O1,T1
3,1_D1_V1_O1_T1,4,Postavljanje poprecne stranice,23.6,28.8,5.2,1,O1,T1
4,1_D1_V1_O1_T1,7,Umetanje i ucvrscivanje kopce_DG,29.0,35.0,6.0,1,O1,T1
...,...,...,...,...,...,...,...,...,...
5575,628_D7_V22_O3_T1,7,Umetanje i ucvrscivanje kopce_DG,29.2,36.2,7.0,628,O3,T1
5576,628_D7_V22_O3_T1,5,Postavljanje uzduzne stranice,36.6,39.2,2.6,628,O3,T1
5577,628_D7_V22_O3_T1,8,Umetanje i ucvrscivanje kopce_DD,39.6,47.0,7.4,628,O3,T1
5578,628_D7_V22_O3_T1,9,Umetanje i ucvrscivanje kopce_LD,48.4,54.0,5.6,628,O3,T1


**Zaključak:** Na temelju analize, umjesto inicijalnih 6 oznaka (pozadina nije uračunata), podatci će biti označeni sa 9 oznaka. Razlog tome su statistički značajne razlike između 4 različite lokacije na kojima obavlja aktivnost 2.